In [1]:
import openpyxl
import csv
import json
import requests
import re
import pandas as pd
import time
from time import sleep
import pyautogui as pag
import undetected_chromedriver as uc
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import Edge
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchAttributeException

In [14]:
options = uc.ChromeOptions()
options.headless = False
options.add_argument("--disable-blink-features=AutomationControlled")
driver = uc.Chrome(version_main=133)
driver.get("https://www.booking.com/searchresults.vi.html?ss=Ti%E1%BB%81n+Giang%2C+Vi%E1%BB%87t+Nam&efdco=1&label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9213289%3Ali%3Adec%3Adm&aid=2311236&lang=vi&sb=1&src_elem=sb&src=index&dest_id=6887&dest_type=region&ac_position=2&ac_click_type=b&ac_langcode=vi&ac_suggestion_list_length=5&search_selected=true&search_pageview_id=9fc222b367620146&ac_meta=GhA5ZmMyMjJiMzY3NjIwMTQ2IAIoATICdmk6B1Rp4buBbiBAAEoAUAA%3D&checkin=2025-03-19&checkout=2025-03-20&group_adults=2&no_rooms=1&group_children=0")

In [33]:
page_source = BeautifulSoup(driver.page_source, 'html.parser')

In [34]:
hotel_div = page_source.find_all('h3', class_='aab71f8e4e')
hotel_URL = []
for hotel in hotel_div:
    a_tag = hotel.find('a')
    if a_tag and a_tag.get('href'):
        href = a_tag.get('href')
        hotel_URL.append(href)
print(len(hotel_URL))
print(hotel_URL)

753
['https://www.booking.com/hotel/vn/muong-thanh-vien-trieu-nha-trang-03-pham-van-dong.vi.html?label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9213289%3Ali%3Adec%3Adm&aid=2311236&ucfs=1&arphpl=1&checkin=2025-02-26&checkout=2025-02-27&dest_id=5425&dest_type=region&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&srpvid=2141373eeae80412&srepoch=1739519618&all_sr_blocks=1009268101_373636079_2_42_0&highlighted_blocks=1009268101_373636079_2_42_0&matching_block_id=1009268101_373636079_2_42_0&sr_pri_blocks=1009268101_373636079_2_42_0__86877500&from=searchresults', 'https://www.booking.com/hotel/vn/muong-thanh-nha-trang-apartment-review-nha-trang.vi.html?label=vi-vn-booking-desktop-ztDRHDT0cyr*llk1cdAHaQS652796014482%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9213289%3Ali%3Adec%3Adm&aid=2311236&ucfs=1&arphpl=1&checkin=2025-02-26&checkout=20

In [36]:
list_name = []
list_address = []
list_latitude = []
list_longitude = []
total_url = len(hotel_URL)
for index, hotel in enumerate(hotel_URL):
    try:
        response = requests.get(hotel, timeout=10)
        # time.sleep(1)
        soup = BeautifulSoup(response.text, 'html.parser')
        hotel_name = soup.find("h2", class_= "d2fee87262 pp-header__title").text.strip()
        hotel_address = soup.find("div", class_= "a53cbfa6de f17adf7576")
        if hotel_address:
            for child in hotel_address.find_all(recursive=False):
                child.extract()
            hotel_address = hotel_address.text.strip()
        list_name.append(hotel_name)
        list_address.append(hotel_address)
        script_tags = soup.find_all('script', nonce=True)
        for script_tag in script_tags:
            if script_tag.string and 'b_map_center_latitude' in script_tag.string:
                script_content = script_tag.string
                lat_match = re.search(r"b_map_center_latitude\s*=\s*([0-9.\-]+);", script_content)
                lon_match = re.search(r"b_map_center_longitude\s*=\s*([0-9.\-]+);", script_content)

                if lat_match and lon_match:
                    latitude = float(lat_match.group(1))
                    longitude = float(lon_match.group(1))
                break            
        if latitude and longitude:
            list_latitude.append(latitude)
            list_longitude.append(longitude)
            print(f"Index:{index+1}/{total_url} - Name:{hotel_name} - Address:{hotel_address} - Lat:{latitude} - Long:{longitude}")
        else:
            print("Không tìm thấy latitude hoặc longitude.")
    except Exception as e:
        print(f"Lỗi khi xử lý URL {hotel}: {e}")

Index:1/753 - Name:Nha Trang Beach Luxury Apartment - Address:03 - 05 Phạm Văn Đồng, Vĩnh Phước, Nha trang, Nha Trang, Việt Nam - Lat:12.27370799 - Long:109.20241138
Index:2/753 - Name:Oceanus MT Vien Trieu Thai Ha Hotel & Apartment - Address:03 Phạm Văn Đồng, Nha Trang, Việt Nam - Lat:12.27450345 - Long:109.2017634
Index:3/753 - Name:Putin Nha Trang Hotel - Address:Bãi Dương 6B, Nha Trang, Việt Nam - Lat:12.27573235 - Long:109.20171881
Index:4/753 - Name:Apus Hotel - Address:95/9 Hung Vuong Street, Loc Tho Ward, Nha Trang, Việt Nam - Lat:12.23446255 - Long:109.19619977
Index:5/753 - Name:Prime New Hotel - Address:22 Ton Dan, Nha Trang, Việt Nam - Lat:12.23751339 - Long:109.19619173
Index:6/753 - Name:Bonjour Nha Trang Hotel - Address:17/2A Nguyen Thi Minh Khai, Nha Trang, Việt Nam - Lat:12.23836694 - Long:109.19438342
Index:7/753 - Name:Pavillon Boutique Hotel & Apartment Nha Trang - Address:100/10B Tran Phu Street, Loc Tho Ward, Nha Trang, Việt Nam - Lat:12.2264158 - Long:109.

In [40]:
print(len(hotel))
print(len(list_name))
print(len(list_address))
print(len(list_latitude))
print(len(list_longitude))

770
752
752
752
752


In [37]:
df = pd.DataFrame({
    'Hotel Name': list_name,
    'Address': list_address,
    'Latitude': list_latitude,
    'Longitude': list_longitude,
    'Link': hotel_URL
})
df.to_csv("D:/TRUNG/Booking/Khánh Hòa Booking/Booking_KhanhHoa.csv", index= False, encoding='utf-8-sig')
display(df)

ValueError: All arrays must be of the same length